# Explore here

In [73]:
#cargamos ficheros en dos dataframes de pandas
import pandas as pd

# URLs de los archivos CSV
movies_url = '/Users/luiseduardogarciablanco/Desktop/bootcamp/KNN/kNN_project/tmdb_5000_movies.csv'
credits_url = '/Users/luiseduardogarciablanco/Desktop/bootcamp/KNN/kNN_project/tmdb_5000_credits.csv'
# Cargar los datos en DataFrames
movies_df = pd.read_csv(movies_url)
credits_df = pd.read_csv(credits_url)

# Mostrar las primeras filas de cada DataFrame para asegurarnos de que se cargaron correctamente
print(movies_df.head())
print(credits_df.head())
movies_df.shape
credits_df.shape

      budget                                             genres  \
0  237000000  [{"id": 28, "name": "Action"}, {"id": 12, "nam...   
1  300000000  [{"id": 12, "name": "Adventure"}, {"id": 14, "...   
2  245000000  [{"id": 28, "name": "Action"}, {"id": 12, "nam...   
3  250000000  [{"id": 28, "name": "Action"}, {"id": 80, "nam...   
4  260000000  [{"id": 28, "name": "Action"}, {"id": 12, "nam...   

                                       homepage      id  \
0                   http://www.avatarmovie.com/   19995   
1  http://disney.go.com/disneypictures/pirates/     285   
2   http://www.sonypictures.com/movies/spectre/  206647   
3            http://www.thedarkknightrises.com/   49026   
4          http://movies.disney.com/john-carter   49529   

                                            keywords original_language  \
0  [{"id": 1463, "name": "culture clash"}, {"id":...                en   
1  [{"id": 270, "name": "ocean"}, {"id": 726, "na...                en   
2  [{"id": 470, "nam

(4803, 4)

In [51]:
# Renombrar la columna 'id' a 'movie_id' en movies_df si es necesario
if 'id' in movies_df.columns:
    movies_df.rename(columns={'id': 'movie_id'}, inplace=True)

Crearemos una base de datos SQLite e insertamos los DataFrames en tablas. Después, unimos las dos tablas utilizando el título de la película como Key.

In [52]:
import sqlite3

# Crear una conexión a una base de datos SQLite
conn = sqlite3.connect('movies.db')

# Escribir los DataFrames en tablas de la base de datos
movies_df.to_sql('movies', conn, if_exists='replace', index=False)
credits_df.to_sql('credits', conn, if_exists='replace', index=False)

# Asegurarse de que las tablas se han creado correctamente
print(pd.read_sql('SELECT * FROM movies LIMIT 5', conn))
print(pd.read_sql('SELECT * FROM credits LIMIT 5', conn))

# Unir las dos tablas usando el título de la película
query = '''
SELECT m.*, c.cast, c.crew
FROM movies m
JOIN credits c ON m.title = c.title
'''

# Crear un DataFrame con la tabla unida
combined_df = pd.read_sql(query, conn)

# Mostrar las primeras filas del DataFrame combinado
print(combined_df.head())

# Cerrar la conexión
conn.close()

      budget                                             genres  \
0  237000000  [{"id": 28, "name": "Action"}, {"id": 12, "nam...   
1  300000000  [{"id": 12, "name": "Adventure"}, {"id": 14, "...   
2  245000000  [{"id": 28, "name": "Action"}, {"id": 12, "nam...   
3  250000000  [{"id": 28, "name": "Action"}, {"id": 80, "nam...   
4  260000000  [{"id": 28, "name": "Action"}, {"id": 12, "nam...   

                                       homepage  movie_id  \
0                   http://www.avatarmovie.com/     19995   
1  http://disney.go.com/disneypictures/pirates/       285   
2   http://www.sonypictures.com/movies/spectre/    206647   
3            http://www.thedarkknightrises.com/     49026   
4          http://movies.disney.com/john-carter     49529   

                                            keywords original_language  \
0  [{"id": 1463, "name": "culture clash"}, {"id":...                en   
1  [{"id": 270, "name": "ocean"}, {"id": 726, "na...                en   
2  [{"id

In [100]:

# Seleccionar solo las columnas deseadas
columns_to_keep = ['movie_id', 'title', 'overview', 'genres', 'keywords', 'cast', 'crew']
combined_df = combined_df[columns_to_keep]
combined_df_emb = combined_df

# Mostrar las primeras filas del DataFrame limpio
print(combined_df.columns)

Index(['movie_id', 'title', 'overview', 'genres', 'keywords', 'cast', 'crew'], dtype='object')


### Paso 3: Transformar los datos

Hay algunas columnas con formato JSON. De cada uno de los JSONs, seleccionamos el atributo `name` y reemplazamos las columnas `genres` y `keywords`. Para la columna `cast`, seleccionando los tres primeros nombres.

In [25]:
import json
# Función para extraer 'name' de una columna JSON
def extract_names(json_str):
    try:
        json_list = json.loads(json_str)
        names = [d['name'] for d in json_list]
        return ', '.join(names)
    except (TypeError, json.JSONDecodeError):
        return ''

# Aplicar la función a las columnas 'genres' y 'keywords'
combined_df['genres'] = combined_df['genres'].apply(extract_names)
combined_df['keywords'] = combined_df['keywords'].apply(extract_names)

# Función para extraer los tres primeros nombres del elenco
def extract_cast_names(json_str):
    try:
        json_list = json.loads(json_str)
        names = [d['name'] for d in json_list[:3]]
        return ', '.join(names)
    except (TypeError, json.JSONDecodeError):
        return ''

# Aplicar la función a la columna 'cast'
combined_df['cast'] = combined_df['cast'].apply(extract_cast_names)

# Mostrar las primeras filas del DataFrame transformado
print(combined_df.head())

   movie_id                                     title  \
0     19995                                    Avatar   
1       285  Pirates of the Caribbean: At World's End   
2    206647                                   Spectre   
3     49026                     The Dark Knight Rises   
4     49529                               John Carter   

                                            overview  \
0  In the 22nd century, a paraplegic Marine is di...   
1  Captain Barbossa, long believed to be dead, ha...   
2  A cryptic message from Bond’s past sends him o...   
3  Following the death of District Attorney Harve...   
4  John Carter is a war-weary, former military ca...   

                                        genres  \
0  Action, Adventure, Fantasy, Science Fiction   
1                   Adventure, Fantasy, Action   
2                     Action, Adventure, Crime   
3               Action, Crime, Drama, Thriller   
4           Action, Adventure, Science Fiction   

                   

Las únicas columnas que quedan por modificar son `crew` (equipo) y `overview` (resumen). Para la primera columna, la transformamos para que contenga el nombre del director. Para la segunda, la convertimos en una lista.

In [26]:
# Función para extraer el nombre del director de la columna 'crew'
def extract_director(json_str):
    try:
        json_list = json.loads(json_str)
        for d in json_list:
            if d['job'] == 'Director':
                return d['name']
        return ''
    except (TypeError, json.JSONDecodeError):
        return ''

# Aplicar la función a la columna 'crew'
combined_df['crew'] = combined_df['crew'].apply(extract_director)

# Convertir la columna 'overview' en una lista de palabras
combined_df['overview'] = combined_df['overview'].apply(lambda x: x.split() if isinstance(x, str) else [])

# Mostrar las primeras filas del DataFrame transformado
print(combined_df.head())

   movie_id                                     title  \
0     19995                                    Avatar   
1       285  Pirates of the Caribbean: At World's End   
2    206647                                   Spectre   
3     49026                     The Dark Knight Rises   
4     49529                               John Carter   

                                            overview  \
0  [In, the, 22nd, century,, a, paraplegic, Marin...   
1  [Captain, Barbossa,, long, believed, to, be, d...   
2  [A, cryptic, message, from, Bond’s, past, send...   
3  [Following, the, death, of, District, Attorney...   
4  [John, Carter, is, a, war-weary,, former, mili...   

                                        genres  \
0  Action, Adventure, Fantasy, Science Fiction   
1                   Adventure, Fantasy, Action   
2                     Action, Adventure, Crime   
3               Action, Crime, Drama, Thriller   
4           Action, Adventure, Science Fiction   

                   

Una vez hayamos terminado de procesar las columnas y que el modelo de recomendación no se confunda, por ejemplo, entre *Jennifer Aniston* y *Jennifer Conelly*, quitaremos los espacios entre las palabras. Aplica esta función a las columnas `genres`, `cast`, `crew` y `keywords`.


In [27]:
import pandas as pd
import json

# Función para extraer 'name' de una columna JSON y eliminar espacios
def extract_names_and_remove_spaces(json_str):
    try:
        json_list = json.loads(json_str)
        names = [d['name'].replace(' ', '') for d in json_list]
        return ', '.join(names)
    except (TypeError, json.JSONDecodeError):
        return ''

# Aplicar la función a las columnas 'genres', 'keywords', 'cast' y 'crew' en combined_df
combined_df['genres'] = combined_df['genres'].apply(extract_names_and_remove_spaces)
combined_df['keywords'] = combined_df['keywords'].apply(extract_names_and_remove_spaces)
combined_df['cast'] = combined_df['cast'].apply(extract_names_and_remove_spaces)

# Función para extraer el nombre del director de la columna 'crew' y eliminar espacios
def extract_director_and_remove_spaces(json_str):
    try:
        json_list = json.loads(json_str)
        for d in json_list:
            if d['job'] == 'Director':
                return d['name'].replace(' ', '')
        return ''
    except (TypeError, json.JSONDecodeError):
        return ''

# Aplicar la función a la columna 'crew' en combined_df
combined_df['crew'] = combined_df['crew'].apply(extract_director_and_remove_spaces)

# Mostrar las primeras filas del DataFrame transformado
print(combined_df.head())

   movie_id                                     title  \
0     19995                                    Avatar   
1       285  Pirates of the Caribbean: At World's End   
2    206647                                   Spectre   
3     49026                     The Dark Knight Rises   
4     49529                               John Carter   

                                            overview genres keywords cast crew  
0  [In, the, 22nd, century,, a, paraplegic, Marin...                            
1  [Captain, Barbossa,, long, believed, to, be, d...                            
2  [A, cryptic, message, from, Bond’s, past, send...                            
3  [Following, the, death, of, District, Attorney...                            
4  [John, Carter, is, a, war-weary,, former, mili...                            


In [28]:
combined_df.columns


Index(['movie_id', 'title', 'overview', 'genres', 'keywords', 'cast', 'crew'], dtype='object')

Por último, reduciremos nuestro conjunto de datos combinando todas nuestras columnas convertidas anteriores en una sola columna llamada `tags` (que crearemos)

In [29]:
# Convertir todas las columnas relevantes a cadenas de texto si no lo están ya
combined_df['overview'] = combined_df['overview'].astype(str)
combined_df['genres'] = combined_df['genres'].astype(str)
combined_df['keywords'] = combined_df['keywords'].astype(str)
combined_df['cast'] = combined_df['cast'].astype(str)
combined_df['crew'] = combined_df['crew'].astype(str)

# Crear la columna 'tags' combinando todas las columnas convertidas
combined_df['tags'] = combined_df['overview'] + ' ' + \
                      combined_df['genres'] + ' ' + \
                      combined_df['keywords'] + ' ' + \
                      combined_df['cast'] + ' ' + \
                      combined_df['crew']

# Reemplazar las comas por espacios en blanco en la columna 'tags'
combined_df['tags'] = combined_df['tags'].apply(lambda x: x.replace(',', ' '))

# Eliminar las comillas simples de cada palabra individual en la columna 'tags'
combined_df['tags'] = combined_df['tags'].apply(lambda x: ' '.join([word.strip("'") for word in x.split()]))

# Mostrar las primeras filas de la columna 'tags' en el DataFrame
print(combined_df['tags'].head())

0    ['In the 22nd century  a paraplegic Marine is ...
1    ['Captain Barbossa  long believed to be dead  ...
2    ['A cryptic message from Bond’s past sends him...
3    ['Following the death of District Attorney Har...
4    ['John Carter is a war-weary  former military ...
Name: tags, dtype: object


In [30]:
combined_df.shape

(4809, 8)

ya con el "EDA" completado procedemos a construir el KNN

In [39]:
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.neighbors import NearestNeighbors

# Paso de preprocesamiento y vectorización
tfidf_vectorizer = TfidfVectorizer(stop_words='english')
#tfidf_vectorizer = TfidfVectorizer(stop_words='english', max_df=0.8, min_df=0.02, ngram_range=(1, 2)) #intentamos mejorar los valores de recomendacion 
tfidf_matrix = tfidf_vectorizer.fit_transform(combined_df['tags'])

# Mostrar el tamaño de la matriz TF-IDF (número de películas, número de términos únicos)
print("Shape of TF-IDF matrix:", tfidf_matrix.shape)

Shape of TF-IDF matrix: (4809, 20979)


In [40]:
from sklearn.metrics.pairwise import cosine_similarity



# Calcular la matriz de similitud utilizando cosine_similarity
similarity_matrix = cosine_similarity(tfidf_matrix, tfidf_matrix)

# Mostrar la forma de la matriz de similitud (número de películas x número de películas)
print("Shape of similarity matrix:", similarity_matrix.shape)

# Mostrar la matriz de similitud
print("Similarity matrix:")
print(similarity_matrix)

Shape of similarity matrix: (4809, 4809)
Similarity matrix:
[[1.         0.         0.         ... 0.         0.         0.        ]
 [0.         1.         0.         ... 0.02161047 0.         0.        ]
 [0.         0.         1.         ... 0.01489056 0.         0.        ]
 ...
 [0.         0.02161047 0.01489056 ... 1.         0.01607413 0.00702169]
 [0.         0.         0.         ... 0.01607413 1.         0.01171699]
 [0.         0.         0.         ... 0.00702169 0.01171699 1.        ]]


In [43]:
#A continuación, construiremos el modelo KNN usando la matriz TF-IDF que hemos generado.
from sklearn.neighbors import NearestNeighbors

model_knn = NearestNeighbors(n_neighbors=7, algorithm='brute', metric='cosine')
model_knn.fit(tfidf_matrix)

NearestNeighbors(algorithm='brute', metric='cosine', n_neighbors=7)

In [44]:


# Calcular la matriz de similitud utilizando cosine_similarity
similarity_matrix = cosine_similarity(tfidf_matrix, tfidf_matrix)

# Mostrar la forma de la matriz de similitud (número de películas x número de películas)
print("Shape of similarity matrix:", similarity_matrix.shape)

# Mostrar la matriz de similitud para verificar el número de películas y mostramos la matriz para revisar los valores de similitud entre los pares de películas.
print("Similarity matrix:")
print(similarity_matrix)

Shape of similarity matrix: (4809, 4809)
Similarity matrix:
[[1.         0.         0.         ... 0.         0.         0.        ]
 [0.         1.         0.         ... 0.02161047 0.         0.        ]
 [0.         0.         1.         ... 0.01489056 0.         0.        ]
 ...
 [0.         0.02161047 0.01489056 ... 1.         0.01607413 0.00702169]
 [0.         0.         0.         ... 0.01607413 1.         0.01171699]
 [0.         0.         0.         ... 0.00702169 0.01171699 1.        ]]


In [138]:
def recommend(movie):
    # Obtener el índice de la película en el DataFrame new_df
    movie_index = combined_df[combined_df["title"] == movie].index[0]
    
    # Obtener las distancias de similitud con respecto a la película dada
    distances = similarity_matrix[movie_index]
    
    # Obtener los índices de las películas más similares, excluyendo la película dada
    movie_list = sorted(list(enumerate(distances)), reverse=True, key=lambda x: x[1])[1:6]
    
    # Imprimir las películas recomendadas
    print(f"Recomendaciones para '{movie}':")
    for i, (index, sim) in enumerate(movie_list, 1):
        print(f"{i}: {combined_df.iloc[index]['title']}")

# Ejemplo de uso de la función recommend, copiar y pegar el nombre de la pelicula exactamente como aparece en la lista original para comprobar recomendaciones 
recommend("Batman Begins")

Recomendaciones para 'Batman Begins':
1: Batman Forever
2: Batman
3: Batman
4: Batman Returns
5: The Dark Knight Rises


Se ha exportado la columna original_title del dataframe original, para saber el nombre exacto de las peliculas y asi poder comprobar el modelo, abre el csv llamado titles_movies.csv y copia el nombre de la pelicula para comprobar las recomendaciones

Utilizamos Metodos de Embadding para intentar nuevos resultados

In [119]:
#Modelos Preentrenados de transformer
import pandas as pd
import numpy as np
import torch
from transformers import AutoTokenizer, AutoModel
from sklearn.metrics.pairwise import cosine_similarity
import json

In [120]:

# Configurar el dispositivo para usar Metal Performance Shaders (MPS) si está disponible
device = torch.device('cpu')

# Función para generar embeddings
def generate_embedding(text, tokenizer, model):
    inputs = tokenizer(text, return_tensors='pt', truncation=True, padding=True, max_length=512)
    with torch.no_grad():
        outputs = model(**inputs.to(device))
    return outputs.last_hidden_state.mean(dim=1).squeeze().cpu().numpy()

# Convertir columnas que pueden ser listas a cadenas unidas por espacios
def convert_to_string(x):
    if isinstance(x, list):
        return ' '.join(x)
    else:
        return str(x)

# Aplicar la conversión a las columnas necesarias
columns_to_convert = ['overview', 'genres', 'keywords', 'cast', 'crew']
for col in columns_to_convert:
    combined_df_emb[col] = combined_df_emb[col].apply(convert_to_string)

# Combinar todas las características textuales en una única columna
combined_df_emb['combined_features'] = combined_df_emb.apply(
    lambda row: row['overview'] + ' ' + row['genres'] + ' ' + row['keywords'] + ' ' + row['cast'] + ' ' + row['crew'],
    axis=1
)

# Obtener los textos
texts = combined_df_emb['combined_features'].tolist()
titles = combined_df_emb['title'].tolist()



In [121]:

# Cargar el modelo y tokenizador de transformers
model_name = 'sentence-transformers/all-MiniLM-L6-v2'  # Puedes usar otro modelo preentrenado si lo prefieres
tokenizer = AutoTokenizer.from_pretrained(model_name)
model = AutoModel.from_pretrained(model_name).to(device)

In [122]:

# Generar embeddings
import time
start_time = time.time()
embeddings = []
for text in texts:
    embedding = generate_embedding(text, tokenizer, model)
    embeddings.append(embedding)
end_time = time.time()
print(f"Tiempo de generación de embeddings: {end_time - start_time} segundos")


Tiempo de generación de embeddings: 206.9860908985138 segundos


In [132]:
# Guardar embeddings en un archivo JSON
embeddings_data = []
for title, embedding in zip(titles, embeddings):
    embeddings_data.append({
        'title': title,
        'embedding': embedding.tolist()  # Convertir numpy array a lista para JSON
    })

output_file = 'embeddings.json'
with open(output_file, 'w') as f:
    for entry in embeddings_data:
        json.dump(entry, f)
        f.write('\n')

# Leer embeddings desde el archivo JSON
embeddings_dict = {}
with open(output_file, 'r') as f:
    for line in f:
        entry = json.loads(line)
        embeddings_dict[entry['title']] = np.array(entry['embedding'])

In [133]:
# Función para encontrar películas similares basadas en la similitud del coseno
def find_similar_movies(movie_title, embeddings_dict, top_n=5):
    if movie_title not in embeddings_dict:
        return f"Movie title '{movie_title}' not found in the dataset."
    movie_embedding = embeddings_dict[movie_title].reshape(1, -1)
    similarities = []
    for title, embedding in embeddings_dict.items():
        if title != movie_title:
            sim = cosine_similarity(movie_embedding, embedding.reshape(1, -1)).flatten()[0]
            similarities.append((title, sim))
    # Ordenar por puntaje de similitud
    similarities = sorted(similarities, key=lambda x: x[1], reverse=True)
    return similarities[:top_n]


In [137]:
# Ejemplo de uso
movie_title = "Batman Begins"
similar_movies = find_similar_movies(movie_title, embeddings_dict)
print(f"Top 5 movies similar to '{movie_title}':")
for movie, score in similar_movies:
    print(f"{movie}: {score:.4f}")

Top 5 movies similar to 'Batman Begins':
The Dark Knight: 0.9705
The Dark Knight Rises: 0.9636
New Nightmare: 0.9501
Batman Forever: 0.9485
Face/Off: 0.9443
